In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/')

import os.path
# orig_cwd = os.getcwd()
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
path = '/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/'
os.chdir(path)
import numpy as np
import pandas as pd
import datetime as dt
import sys
import time
from collections import OrderedDict
import importlib
import importlib.util
import joblib
# os.chdir(orig_cwd)


Mounted at /content/drive


## DataPreprocess

In [ ]:
pd.read_csv(dataBefore,index_col =0)

,SessionID,Time,ItemID
0,1,2014-04-07T10:51:09.277Z,214536502
1,1,2014-04-07T10:54:09.868Z,214536500
2,1,2014-04-07T10:54:46.998Z,214536506
3,1,2014-04-07T10:57:00.306Z,214577561
4,2,2014-04-07T13:56:37.614Z,214662742
...,...,...,...
99995,31812,2014-04-01T17:13:14.184Z,214662819
99996,31812,2014-04-01T17:13:49.017Z,214836765
99997,31812,2014-04-01T17:14:12.729Z,214836073
99998,31812,2014-04-01T17:14:48.288Z,214662819


In [32]:
aa = pd.read_csv('/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/data/yoochoose-data/kaggle/yoochoose-clicks_tiny.dat'
, sep=',', usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})

aa.dtypes
aa

,0,1,2
0,1,2014-04-07T10:51:09.277Z,214536502
1,1,2014-04-07T10:54:09.868Z,214536500
2,1,2014-04-07T10:54:46.998Z,214536506
3,1,2014-04-07T10:57:00.306Z,214577561
4,2,2014-04-07T13:56:37.614Z,214662742
...,...,...,...
99995,31812,2014-04-01T17:13:14.184Z,214662819
99996,31812,2014-04-01T17:13:49.017Z,214836765
99997,31812,2014-04-01T17:14:12.729Z,214836073
99998,31812,2014-04-01T17:14:48.288Z,214662819


In [35]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 10 09:50:45 2019
@author: s-moh
"""
import numpy as np
import pandas as pd
import datetime

dataBefore = '/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/data/yoochoose-data/kaggle/yoochoose-clicks_tiny.dat' #Path to Original Training Dataset "Clicks" File
dataTestBefore = '/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/data/yoochoose-data/kaggle/yoochoose-test_tiny.dat' #Path to Original Testing Dataset "Clicks" File
dataAfter ='/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/' #Path to Processed Dataset Folder
dayTime = 86400 #Validation Only one day = 86400 seconds

def removeShortSessions(data):
    #delete sessions of length < 1
    sessionLen = data.groupby('SessionID').size() #group by sessionID and get size of each session
    data = data[np.in1d(data.SessionID, sessionLen[sessionLen > 1].index)]
    return data

#Read Dataset in pandas Dataframe (Ignore Category Column)
train = pd.read_csv(dataBefore, sep=',', usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
test = pd.read_csv(dataTestBefore, sep=',', usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
train.columns = ['SessionID', 'Time', 'ItemID'] #Headers of dataframe
test.columns = ['SessionID', 'Time', 'ItemID'] #Headers of dataframe
train['Time']= train.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #Convert time objects to timestamp
test['Time'] = test.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #Convert time objects to timestamp

#remove sessions of less than 2 interactions
train = removeShortSessions(train)
#delete records of items which appeared less than 5 times
itemLen = train.groupby('ItemID').size() #groupby itemID and get size of each item
train = train[np.in1d(train.ItemID, itemLen[itemLen > 4].index)]
#remove sessions of less than 2 interactions again
train = removeShortSessions(train)

######################################################################################################3
'''
#Separate Data into Train and Test Splits
timeMax = data.Time.max() #maximum time in all records
sessionMaxTime = data.groupby('SessionID').Time.max() #group by sessionID and get the maximum time of each session
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last day
sessionTest  = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #testing split is all sessions has records in the last day
train = data[np.in1d(data.SessionID, sessionTrain)]
test = data[np.in1d(data.SessionID, sessionTest)]
'''
#Delete records in testing split where items are not in training split
test = test[np.in1d(test.ItemID, train.ItemID)]
#Delete Sessions in testing split which are less than 2
test = removeShortSessions(test)

#Convert To CSV
#print('Full Training Set has', len(train), 'Events, ', train.SessionID.nunique(), 'Sessions, and', train.ItemID.nunique(), 'Items\n\n')
#train.to_csv(dataAfter + 'recSys15TrainFull.txt', sep='\t', index=False)
print('Testing Set has', len(test), 'Events, ', test.SessionID.nunique(), 'Sessions, and', test.ItemID.nunique(), 'Items\n\n')
test.to_csv(dataAfter + 'recSys15Test.txt', sep=',', index=False)

######################################################################################################3
#Separate Training set into Train and Validation Splits
timeMax = train.Time.max()
sessionMaxTime = train.groupby('SessionID').Time.max()
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last 2nd day
sessionValid = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #validation split is all sessions that ended during the last 2nd day
trainTR = train[np.in1d(train.SessionID, sessionTrain)]
trainVD = train[np.in1d(train.SessionID, sessionValid)]
#Delete records in validation split where items are not in training split
trainVD = trainVD[np.in1d(trainVD.ItemID, trainTR.ItemID)]
#Delete Sessions in testing split which are less than 2
trainVD = removeShortSessions(trainVD)
#Convert To CSV
print('Training Set has', len(trainTR), 'Events, ', trainTR.SessionID.nunique(), 'Sessions, and', trainTR.ItemID.nunique(), 'Items\n\n')
trainTR.to_csv(dataAfter + 'recSys15TrainOnly.txt', sep=',', index=False)
print('Validation Set has', len(trainVD), 'Events, ', trainVD.SessionID.nunique(), 'Sessions, and', trainVD.ItemID.nunique(), 'Items\n\n')
trainVD.to_csv(dataAfter + 'recSys15Valid.txt', sep=',', index=False)

Testing Set has 79483 Events,  20672 Sessions, and 2782 Items


Training Set has 70278 Events,  17794 Sessions, and 2933 Items


Validation Set has 13568 Events,  3416 Sessions, and 1771 Items




## Model Run

In [36]:
import easydict

args = easydict.EasyDict({
        "hidden_size": 100,
        "num_layers": 3,
        "batch_size": 50,
        "dropout_input": 0,
        "dropout_hidden": 0.5,
        "n_epochs": 5,
        "k_eval": 20,
        "optimizer_type":'Adagrad',
        "final_act": 'tanh',
        "lr": 0.01,
        "weight_decay" : 0,
        "momentum":0,
        "eps":1e-6,
        "seed":22,
        "sigma": None,
        "embedding_dim":-1,
        "loss_type": "TOP1-max",
        "time_sort":False,
        "save_dir":"models",
        "data_folder":'/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/',
        "train_data": 'recSys15TrainOnly.txt',
        'valid_data':'recSys15Valid.txt',
        'is_eval': None , # 'store_true',
        'load_model': None,
        "checkpoint_dir": "checkpoint"
})

In [37]:
import argparse
import torch
import lib
import numpy as np
import os
import datetime


args.cuda = torch.cuda.is_available()
#use random seed defined
np.random.seed(args.seed)
torch.manual_seed(args.seed)


if args.cuda:
    torch.cuda.manual_seed(args.seed)

#Write Checkpoints with arguments used in a text file for reproducibility
def make_checkpoint_dir():
    print("PARAMETER" + "-"*10)
    now = datetime.datetime.now()
    S = '{:02d}{:02d}{:02d}{:02d}'.format(now.month, now.day, now.hour, now.minute)
    save_dir = os.path.join(args.checkpoint_dir, S)
    if not os.path.exists(args.checkpoint_dir):
        os.mkdir(args.checkpoint_dir)

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    args.checkpoint_dir = save_dir
    with open(os.path.join(args.checkpoint_dir, 'parameter.txt'), 'w') as f:
        for attr, value in sorted(args.__dict__.items()):
            print("{}={}".format(attr.upper(), value))
            f.write("{}={}\n".format(attr.upper(), value))
    print("---------" + "-"*10)

#weight initialization if it was defined
def init_model(model):
    if args.sigma is not None:
        for p in model.parameters():
            if args.sigma != -1 and args.sigma != -2:
                sigma = args.sigma
                p.data.uniform_(-sigma, sigma)
            elif len(list(p.size())) > 1:
                sigma = np.sqrt(6.0 / (p.size(0) + p.size(1)))
                if args.sigma == -1:
                    p.data.uniform_(-sigma, sigma)
                else:
                    p.data.uniform_(0, sigma)


def main():
    print("Loading train data from {}".format(os.path.join(args.data_folder, args.train_data)))
    print("Loading valid data from {}".format(os.path.join(args.data_folder, args.valid_data)))

    train_data = lib.Dataset(os.path.join(args.data_folder, args.train_data))
    valid_data = lib.Dataset(os.path.join(args.data_folder, args.valid_data), itemmap=train_data.itemmap)
    make_checkpoint_dir()

    #set all the parameters according to the defined arguments
    input_size = len(train_data.items)
    hidden_size = args.hidden_size
    num_layers = args.num_layers
    output_size = input_size
    batch_size = args.batch_size
    dropout_input = args.dropout_input
    dropout_hidden = args.dropout_hidden
    embedding_dim = args.embedding_dim
    final_act = args.final_act
    loss_type = args.loss_type
    optimizer_type = args.optimizer_type
    lr = args.lr
    weight_decay = args.weight_decay
    momentum = args.momentum
    eps = args.eps
    n_epochs = args.n_epochs
    time_sort = args.time_sort
    #loss function
    loss_function = lib.LossFunction(loss_type=loss_type, use_cuda=args.cuda) #cuda is used with cross entropy only
    if not args.is_eval: #training
        #Initialize the model
        model = lib.GRU4REC(input_size, hidden_size, output_size, final_act=final_act,
                            num_layers=num_layers, use_cuda=args.cuda, batch_size=batch_size,
                            dropout_input=dropout_input, dropout_hidden=dropout_hidden, embedding_dim=embedding_dim)
        #weights initialization
        init_model(model)
        #optimizer
        optimizer = lib.Optimizer(model.parameters(), optimizer_type=optimizer_type, lr=lr,
                                  weight_decay=weight_decay, momentum=momentum, eps=eps)
        #trainer class
        trainer = lib.Trainer(model, train_data=train_data, eval_data=valid_data, optim=optimizer,
                              use_cuda=args.cuda, loss_func=loss_function, batch_size=batch_size, args=args)
        print('#### START TRAINING....')
        trainer.train(0, n_epochs - 1)
    else: #testing
        if args.load_model is not None:
            print("Loading pre-trained model from {}".format(args.load_model))
            try:
                checkpoint = torch.load(args.load_model)
            except:
                checkpoint = torch.load(args.load_model, map_location=lambda storage, loc: storage)
            model = checkpoint["model"]
            model.gru.flatten_parameters()
            evaluation = lib.Evaluation(model, loss_function, use_cuda=args.cuda, k = args.k_eval)
            loss, recall, mrr = evaluation.eval(valid_data, batch_size)
            print("Final result: recall = {:.2f}, mrr = {:.2f}".format(recall, mrr))
        else:
            print("No Pretrained Model was found!")


if __name__ == '__main__':
    main()

Loading train data from /content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/recSys15TrainOnly.txt
Loading valid data from /content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/recSys15Valid.txt
PARAMETER----------
BATCH_SIZE=50
CHECKPOINT_DIR=checkpoint/08121028
CUDA=True
DATA_FOLDER=/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/
DROPOUT_HIDDEN=0.5
DROPOUT_INPUT=0
EMBEDDING_DIM=-1
EPS=1e-06
FINAL_ACT=tanh
HIDDEN_SIZE=100
IS_EVAL=None
K_EVAL=20
LOAD_MODEL=None
LOSS_TYPE=TOP1-max
LR=0.01
MOMENTUM=0
N_EPOCHS=5
NUM_LAYERS=3
OPTIMIZER_TYPE=Adagrad
SAVE_DIR=models
SEED=22
SIGMA=None
TIME_SORT=False
TRAIN_DATA=recSys15TrainOnly.txt
VALID_DATA=recSys15Valid.txt
WEIGHT_DECAY=0
-------------------
#### START TRAINING....

 73%|███████▎  | 199/271 [00:00<00:00, 667.53it/s]


TypeError: ignored

In [ ]:
def split_info(data: pd.DataFrame, status: str):
  print(f'*{status} Data Stats Info\n'
        f'\t Events'




        )

In [ ]:
args

{'hidden_size': 100,
 'num_layers': 3,
 'batch_size': 50,
 'dropout_input': 0,
 'dropout_hidden': 0.5,
 'n_epochs': 5,
 'k_eval': 20,
 'optimizer_type': 'Adagrad',
 'final_act': 'tanh',
 'lr': 0.01,
 'weight_decay': 0,
 'momentum': 0,
 'eps': 1e-06,
 'seed': 22,
 'sigma': None,
 'embedding_dim': -1,
 'loss_type': 'TOP1-max',
 'time_sort': False,
 'save_dir': 'models',
 'data_folder': '/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After/',
 'train_data': 'recSys15TrainOnly.txt',
 'valid_data': 'recSys15Valid.txt',
 'is_eval': 'store_true',
 'load_model': None,
 'checkpoint_dir': 'checkpoint/08120821',
 'cuda': False}

In [ ]:
import argparse
import torch
import lib
import numpy as np
import os
import datetime

parser = argparse.ArgumentParser()
parser.add_argument('--hidden_size', default=100, type=int) #Literature uses 100 / 1000 --> better is 100
parser.add_argument('--num_layers', default=3, type=int) #1 hidden layer
parser.add_argument('--batch_size', default=50, type=int) #50 in first paper and 32 in second paper
parser.add_argument('--dropout_input', default=0, type=float) #0.5 for TOP and 0.3 for BPR
parser.add_argument('--dropout_hidden', default=0.5, type=float) #0.5 for TOP and 0.3 for BPR
parser.add_argument('--n_epochs', default=5, type=int) #number of epochs (10 in literature)
parser.add_argument('--k_eval', default=20, type=int) #value of K durig Recall and MRR Evaluation
# parse the optimizer arguments
parser.add_argument('--optimizer_type', default='Adagrad', type=str) #Optimizer --> Adagrad is the best according to literature
parser.add_argument('--final_act', default='tanh', type=str) #Final Activation Function
parser.add_argument('--lr', default=0.01, type=float) #learning rate (Best according to literature 0.01 to 0.05)
parser.add_argument('--weight_decay', default=0, type=float) #no weight decay
parser.add_argument('--momentum', default=0, type=float) #no momentum
parser.add_argument('--eps', default=1e-6, type=float) #not used
parser.add_argument("-seed", type=int, default=22, help="Seed for random initialization") #Random seed setting
parser.add_argument("-sigma", type=float, default=None, help="init weight -1: range [-sigma, sigma], -2: range [0, sigma]") # weight initialization [-sigma sigma] in literature

####### TODO: discover this ###########
parser.add_argument("--embedding_dim", type=int, default=-1, help="using embedding")
####### TODO: discover this ###########

# parse the loss type
parser.add_argument('--loss_type', default='TOP1-max', type=str) #type of loss function TOP1 / BPR / TOP1-max / BPR-max
# etc
parser.add_argument('--time_sort', default=False, type=bool) #In case items are not sorted by time stamp
parser.add_argument('--model_name', default='GRU4REC-CrossEntropy', type=str)
parser.add_argument('--save_dir', default='models', type=str)
parser.add_argument('--data_folder', default='/content/drive/MyDrive/커리어/추천알고리즘_스터디/[2023 하반기]추천알고리즘 스터디/code/GRU4Rec/GRU4REC-pytorch-master/GRU4REC-pytorch-master/RecSys_Dataset_After_tiny/', type=str)
parser.add_argument('--train_data', default='recSys15TrainOnly.txt', type=str)
parser.add_argument('--valid_data', default='recSys15Valid.txt', type=str)
parser.add_argument("--is_eval", action='store_true') #should be used during testing and eliminated during training
parser.add_argument('--load_model', default=None,  type=str)
parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')

# Get the arguments
args = parser.parse_args()
args.cuda = torch.cuda.is_available()
#use random seed defined
np.random.seed(args.seed)
torch.manual_seed(args.seed)


if args.cuda:
    torch.cuda.manual_seed(args.seed)

#Write Checkpoints with arguments used in a text file for reproducibility
def make_checkpoint_dir():
    print("PARAMETER" + "-"*10)
    now = datetime.datetime.now()
    S = '{:02d}{:02d}{:02d}{:02d}'.format(now.month, now.day, now.hour, now.minute)
    save_dir = os.path.join(args.checkpoint_dir, S)
    if not os.path.exists(args.checkpoint_dir):
        os.mkdir(args.checkpoint_dir)

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    args.checkpoint_dir = save_dir
    with open(os.path.join(args.checkpoint_dir, 'parameter.txt'), 'w') as f:
        for attr, value in sorted(args.__dict__.items()):
            print("{}={}".format(attr.upper(), value))
            f.write("{}={}\n".format(attr.upper(), value))
    print("---------" + "-"*10)

#weight initialization if it was defined
def init_model(model):
    if args.sigma is not None:
        for p in model.parameters():
            if args.sigma != -1 and args.sigma != -2:
                sigma = args.sigma
                p.data.uniform_(-sigma, sigma)
            elif len(list(p.size())) > 1:
                sigma = np.sqrt(6.0 / (p.size(0) + p.size(1)))
                if args.sigma == -1:
                    p.data.uniform_(-sigma, sigma)
                else:
                    p.data.uniform_(0, sigma)


def main():
    print("Loading train data from {}".format(os.path.join(args.data_folder, args.train_data)))
    print("Loading valid data from {}".format(os.path.join(args.data_folder, args.valid_data)))

    train_data = lib.Dataset(os.path.join(args.data_folder, args.train_data))
    valid_data = lib.Dataset(os.path.join(args.data_folder, args.valid_data), itemmap=train_data.itemmap)
    make_checkpoint_dir()

    #set all the parameters according to the defined arguments
    input_size = len(train_data.items)
    hidden_size = args.hidden_size
    num_layers = args.num_layers
    output_size = input_size
    batch_size = args.batch_size
    dropout_input = args.dropout_input
    dropout_hidden = args.dropout_hidden
    embedding_dim = args.embedding_dim
    final_act = args.final_act
    loss_type = args.loss_type
    optimizer_type = args.optimizer_type
    lr = args.lr
    weight_decay = args.weight_decay
    momentum = args.momentum
    eps = args.eps
    n_epochs = args.n_epochs
    time_sort = args.time_sort
    #loss function
    loss_function = lib.LossFunction(loss_type=loss_type, use_cuda=args.cuda) #cuda is used with cross entropy only
    if not args.is_eval: #training
        #Initialize the model
        model = lib.GRU4REC(input_size, hidden_size, output_size, final_act=final_act,
                            num_layers=num_layers, use_cuda=args.cuda, batch_size=batch_size,
                            dropout_input=dropout_input, dropout_hidden=dropout_hidden, embedding_dim=embedding_dim)
        #weights initialization
        init_model(model)
        #optimizer
        optimizer = lib.Optimizer(model.parameters(), optimizer_type=optimizer_type, lr=lr,
                                  weight_decay=weight_decay, momentum=momentum, eps=eps)
        #trainer class
        trainer = lib.Trainer(model, train_data=train_data, eval_data=valid_data, optim=optimizer,
                              use_cuda=args.cuda, loss_func=loss_function, batch_size=batch_size, args=args)
        print('#### START TRAINING....')
        trainer.train(0, n_epochs - 1)
    else: #testing
        if args.load_model is not None:
            print("Loading pre-trained model from {}".format(args.load_model))
            try:
                checkpoint = torch.load(args.load_model)
            except:
                checkpoint = torch.load(args.load_model, map_location=lambda storage, loc: storage)
            model = checkpoint["model"]
            model.gru.flatten_parameters()
            evaluation = lib.Evaluation(model, loss_function, use_cuda=args.cuda, k = args.k_eval)
            loss, recall, mrr = evaluation.eval(valid_data, batch_size)
            print("Final result: recall = {:.2f}, mrr = {:.2f}".format(recall, mrr))
        else:
            print("No Pretrained Model was found!")


if __name__ == '__main__':
    main()